In [ ]:
# create env
# python3 -m venv prompt-gen-env
# source prompt-gen-env/bin/activate

!pip install langchain
!pip install langchain-community
!pip install langchain-huggingface
!pip install faiss-cpu  # or faiss-gpu if you have a compatible GPU
!pip install pandas
!pip install panel jupyter_bokeh

In [16]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import faiss

In [30]:
import panel as pn
pn.extension()

# Dropdown für Dateiauswahl
file_dropdown = pn.widgets.Select(
    name='Datei auswählen',
    options=[
        'elektromotoren-100.csv',
        'customers-100.csv'
    ]
)

# Textanzeige der Auswahl
selected_file_display = pn.pane.Str('', styles={'font-size': '16px', 'color': 'green'})

# Callback-Funktion bei Auswahl
def update_display(event):
    selected_file_display.object = f"Ausgewählte Datei: {event.new}"

file_dropdown.param.watch(update_display, 'value')

# Anzeigen
pn.Column(
    file_dropdown,
    selected_file_display
).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'f2d0a5d1-d92d-47c2-867a-4928207b8347': {'version…

In [31]:
file_path = file_dropdown.value
data = pd.read_csv(file_path)
data.head()

,Index,Motor_ID,Config_ID,Spannung_V,Leistung_kW,Drehmoment_Nm,Temp_Bereich_C,Einbaudatum,Qualitaetsprüfung,Fertigungsstraße
0,1,EM2025000,CFG345,400,44.6,39.2,-25 – 118,2024-07-06,nein,Linie C
1,2,EM2025001,CFG864,400,45.8,252.3,-3 – 87,2025-03-11,ja,Linie B
2,3,EM2025002,CFG460,400,1.9,287.6,-12 – 122,2024-12-19,nein,Linie A
3,4,EM2025003,CFG716,400,21.8,15.8,-29 – 83,2025-02-22,ja,Linie C
4,5,EM2025004,CFG661,400,10.6,430.9,-5 – 125,2023-08-17,ja,Linie A


In [32]:
# Load documents
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()
docs[0:3]

[Document(metadata={'source': 'elektromotoren-100.csv', 'row': 0}, page_content='Index: 1\nMotor_ID: EM2025000\nConfig_ID: CFG345\nSpannung_V: 400\nLeistung_kW: 44.6\nDrehmoment_Nm: 39.2\nTemp_Bereich_C: -25 – 118\nEinbaudatum: 2024-07-06\nQualitaetsprüfung: nein\nFertigungsstraße: Linie C'),
 Document(metadata={'source': 'elektromotoren-100.csv', 'row': 1}, page_content='Index: 2\nMotor_ID: EM2025001\nConfig_ID: CFG864\nSpannung_V: 400\nLeistung_kW: 45.8\nDrehmoment_Nm: 252.3\nTemp_Bereich_C: -3 – 87\nEinbaudatum: 2025-03-11\nQualitaetsprüfung: ja\nFertigungsstraße: Linie B'),
 Document(metadata={'source': 'elektromotoren-100.csv', 'row': 2}, page_content='Index: 3\nMotor_ID: EM2025002\nConfig_ID: CFG460\nSpannung_V: 400\nLeistung_kW: 1.9\nDrehmoment_Nm: 287.6\nTemp_Bereich_C: -12 – 122\nEinbaudatum: 2024-12-19\nQualitaetsprüfung: nein\nFertigungsstraße: Linie A')]

In [33]:
# Use BGE Embeddings
if file_path == "elektromotoren-100.csv":
    embedding_model_name = "BAAI/bge-m3" # mulitlingual
else:
    embedding_model_name = "BAAI/bge-base-en"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [34]:
# Create FAISS index
dim = len(embeddings.embed_query("Hello world"))
print("embeddings dimension:", dim)
index = faiss.IndexFlatL2(dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

embeddings dimension: 1024


In [35]:
vector_store.add_documents(documents=docs)
doc_id = list(vector_store.docstore._dict.keys())[0]
print(doc_id)
print(vector_store.docstore._dict[doc_id].page_content)

0d984407-b8ca-4fb3-b44e-527d9089ec63
Index: 1
Motor_ID: EM2025000
Config_ID: CFG345
Spannung_V: 400
Leistung_kW: 44.6
Drehmoment_Nm: 39.2
Temp_Bereich_C: -25 – 118
Einbaudatum: 2024-07-06
Qualitaetsprüfung: nein
Fertigungsstraße: Linie C


In [36]:
# # Use Mistral 7B via ctransformers
# llm = CTransformers(
#     model='TheBloke/Mistral-7B-Instruct-v0.1-GGUF',  # GGUF quantized
#     # model_file='mistral-7b-instruct-v0.1.Q4_K_M.gguf',
#     config={'max_new_tokens': 10000, 'temperature': 0.7}
# )

In [37]:
# ---- STEP 1: Set up the prompt template ----
if file_path == "elektromotoren-100.csv":
    system_prompt = (
        "Du bist ein Assistent zur Analyse und Auswertung von Daten zu Elektromotoren. "
        "Nutze die folgenden Teile des abgerufenen Tracing-Datensatzes, um die gestellte "
        "Frage zu beantworten. Wenn du die Antwort nicht sicher weißt, gib an, dass du es "
        "nicht weißt. Antworte in maximal drei Sätzen und halte die Antwort präzise und sachlich."
        "\n\n"
        "{context}"
    )


    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])
    
    # ---- STEP 2: Ask a question and retrieve documents ----
    question = "Wann wurde der Motor mit Config_ID: CFG345 eingebaut?"
else:
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])


    # ---- STEP 2: Ask a question and retrieve documents ----
    question = "which company does Sheryl Baxter work for?"

retriever = vector_store.as_retriever()
docs = retriever.invoke(question)

# # ---- STEP 3: Preview the retrieved context ----
# print("Retrieved documents:\n")
# for i, doc in enumerate(docs):
#     print(f"[{i+1}] {doc.page_content}\n")

# ---- STEP 4: Fill the prompt manually ----
filled_prompt = prompt.format(
    context="\n\n".join([doc.page_content for doc in docs]),
    input=question
)

print("\n\n==== Final Prompt Sent to the LLM ====\n")
print(filled_prompt)

# # ---- STEP 5: Run the LLM with the filled prompt ----

# response = llm.invoke(filled_prompt)
# print("\n\n==== LLM Answer ====\n")
# print(response)



==== Final Prompt Sent to the LLM ====

System: Du bist ein Assistent zur Analyse und Auswertung von Daten zu Elektromotoren. Nutze die folgenden Teile des abgerufenen Tracing-Datensatzes, um die gestellte Frage zu beantworten. Wenn du die Antwort nicht sicher weißt, gib an, dass du es nicht weißt. Antworte in maximal drei Sätzen und halte die Antwort präzise und sachlich.

Index: 1
Motor_ID: EM2025000
Config_ID: CFG345
Spannung_V: 400
Leistung_kW: 44.6
Drehmoment_Nm: 39.2
Temp_Bereich_C: -25 – 118
Einbaudatum: 2024-07-06
Qualitaetsprüfung: nein
Fertigungsstraße: Linie C

Index: 35
Motor_ID: EM2025034
Config_ID: CFG500
Spannung_V: 400
Leistung_kW: 45.4
Drehmoment_Nm: 480.3
Temp_Bereich_C: -4 – 95
Einbaudatum: 2024-08-26
Qualitaetsprüfung: ja
Fertigungsstraße: Linie A

Index: 56
Motor_ID: EM2025055
Config_ID: CFG125
Spannung_V: 400
Leistung_kW: 37.7
Drehmoment_Nm: 263.9
Temp_Bereich_C: 0 – 111
Einbaudatum: 2024-04-09
Qualitaetsprüfung: nein
Fertigungsstraße: Linie B

Index: 58
Motor_I